In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('../preview3.csv')

In [3]:
data.head()

,site,title,author,secondary_authors,published_on,accessed_on,url,body,html,newspaper_keywords,newspaper_summary,id
0,HuffingtonPost,Dan de baja a dos 'Centinelas' en universidad,NaN,[],2017-03-31,2017-04-01 10:03:11.663736,http://www.huffingtonpost.com.mx/2017/03/31/ec...,Autoridades de la Procuraduría capitalina exho...,<div><p>Autoridades de la Procuradur&#237;a ca...,"{universidad,la,se,dan,en,centinelas,del,que,l...",Dos jóvenes vinculados con el grupo agresor de...,1946
1,HuffingtonPost,2017년 최고의 만우절 장난을 모아봤다,NaN,[],2017-04-01,2017-04-01 10:03:18.046313,http://www.huffingtonpost.kr/2017/04/01/story_...,게시됨: 2017년 04월 01일 15시 36분 KST 게시됨: 2017년 04월...,<div><p>&#44172;&#49884;&#46120;: 2017&#45380;...,"{모아봤다,만우절,31,2017년,수,한,최고의,국제옐로우피스,장난을,만우절을,속에...",게시됨: 2017년 04월 01일 15시 36분 KST게시됨: 2017년 04월 0...,1947
2,HuffingtonPost,이재명이 첫 2위를 달성했다,NaN,[],2017-03-31,2017-04-01 10:03:24.573691,http://www.huffingtonpost.kr/2017/03/31/story_...,게시됨: 2017년 03월 31일 19시 28분 KST 게시됨: 2017년 03월...,<div><p>&#44172;&#49884;&#46120;: 2017&#45380;...,"{달성했다,저지할,31일,것이라고,이재명이,최선을,후보의,2위를,후보는,첫,할,있다...",이 후보는 이날 부산 동래구 부산실내체육관에서 열린 민주당 영남권역 순회투표 뒤 기...,1948
3,HuffingtonPost,이부진과 이혼소송 중인 임우재 자택 거금에 경매 나왔다,NaN,[],2017-03-31,2017-04-01 10:03:28.095316,http://www.huffingtonpost.kr/2017/03/31/story_...,게시됨: 2017년 03월 31일 20시 07분 KST 게시됨: 2017년 03월...,<div><p>&#44172;&#49884;&#46120;: 2017&#45380;...,"{중인,이혼소송,고문의,하나은행에서,31일,지지옥션은,이부진과,임,임우재,것으로,경...",게시됨: 2017년 03월 31일 20시 07분 KST게시됨: 2017년 03월 3...,1949
4,HuffingtonPost,Bali Kaur Bassi Is The First Punjabi Model On ...,Monika Markovinovic,[],2017-03-31,2017-04-01 10:03:33.762068,http://www.huffingtonpost.ca/2017/03/31/bali-k...,ADVERTISEMENT The 2017 fall runways during fa...,"<div><p class=""sharebox-label"">ADVERTISEMENT</...","{fashion,bassi,bali,post,punjabi,shared,media,...","So finally, one Canadian fashion magazine deci...",1950


## TF IDF outline 

- list of articles bodies 
- feed list into function to make a sparse matrix for training
- use that with tf-idf stuff above to create a tf-idf transformer 
- use that to label every individual article with tf-idf vectors in a database table


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 12 columns):
site                  25 non-null object
title                 25 non-null object
author                15 non-null object
secondary_authors     25 non-null object
published_on          25 non-null object
accessed_on           25 non-null object
url                   25 non-null object
body                  25 non-null object
html                  25 non-null object
newspaper_keywords    25 non-null object
newspaper_summary     25 non-null object
id                    25 non-null int64
dtypes: int64(1), object(11)
memory usage: 2.4+ KB


In [5]:
article_dict = dict(zip(data.id, data.body))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def tf_idf_from_articles(articles_dict):
    '''
        takes in dict of ID, article body
    
        returns a dict with aricle_id, tf_idf vector
    '''
    bodies = [article_dict[id_val] for id_val in article_dict]
 
    bow_trans = CountVectorizer().fit(bodies)
    # get bag of words data for articles
    articles_bow = bow_trans.transform(bodies)
    articles_bow
    tfidf_trans = TfidfTransformer().fit(articles_bow)

    tf_idf_dict = {}

    for uniq_id in article_dict:
        #print(article)
        
        # get article text
        art = article_dict[uniq_id]
        bag_of_words = bow_trans.transform([art])
        tf_idf_vect = tfidf_trans.transform(bag_of_words)
        tf_idf_dict[uniq_id] = tf_idf_vect
    return tf_idf_dict
    #article_1_bow = bow_trans.transform([bodies[0]])

    #tf_idf_test = tfidf_trans.transform(article_1_bow)

    #print(tf_idf_test)

In [7]:
tf_idf_dict = tf_idf_from_articles(article_dict)

In [8]:
for id_val in tf_idf_dict:
    print(id_val)
    print(tf_idf_dict[id_val])

1946
  (0, 3244)	0.0263472187408
  (0, 3236)	0.0263472187408
  (0, 3198)	0.0263472187408
  (0, 3086)	0.0263472187408
  (0, 3085)	0.0263472187408
  (0, 3063)	0.0263472187408
  (0, 3062)	0.0263472187408
  (0, 3058)	0.0263472187408
  (0, 3047)	0.023350576482
  (0, 3046)	0.0182277827044
  (0, 3019)	0.0263472187408
  (0, 2990)	0.0263472187408
  (0, 2976)	0.0526944374816
  (0, 2975)	0.0391505074438
  (0, 2912)	0.0263472187408
  (0, 2908)	0.0263472187408
  (0, 2905)	0.0263472187408
  (0, 2903)	0.0263472187408
  (0, 2900)	0.0526944374816
  (0, 2870)	0.0263472187408
  (0, 2864)	0.0263472187408
  (0, 2855)	0.0263472187408
  (0, 2854)	0.0526944374816
  (0, 2844)	0.0263472187408
  (0, 2812)	0.0526944374816
  :	:
  (0, 426)	0.0263472187408
  (0, 409)	0.0526944374816
  (0, 400)	0.0526944374816
  (0, 395)	0.0526944374816
  (0, 390)	0.0526944374816
  (0, 385)	0.0263472187408
  (0, 369)	0.0263472187408
  (0, 357)	0.0263472187408
  (0, 331)	0.0263472187408
  (0, 318)	0.0263472187408
  (0, 289)	0.0263472

# TODO

- use function with the output of the NLP'd article data
- Iterate over dictionary returned from function and insert each tf-idf vector into table with article id